In [ ]:
from transmap import TransMAP
import pandas as pd
import numpy as np
tm = TransMAP('https://oak.cast.uark.edu/metadata-api')

datasets_AIS = tm.search('"AIS Freight"')
datasets_locks = tm.search('"Locks"')

data_locks= tm.fetch(datasets_locks[0])
Arkansas_locks = data_locks[(data_locks.pmsname == 'WEBBERS FALLS LOCK & DAM') | (data_locks.pmsname == 'W.D. MAYO LOCK & DAM') | (data_locks.pmsname == 'JAMES W. TRIMBLE LOCK & DAM') | (data_locks.pmsname == 'OZARK LOCK & DAM') | (data_locks.pmsname =='DARDANELLE LOCK & DAM') | (data_locks.pmsname =='ARTHUR V. ORMOND L & D')]


In [ ]:
df_2017 = pd.DataFrame()
df_2018 = pd.DataFrame()
df_2019 = pd.DataFrame()

In [ ]:
for lock in Arkansas_locks:
    bbox = [lock[49]-.1,lock[50]-.1,lock[49]+.1,lock[50]+.1]
    ais_2017_by_lock_df = tm.fetch(datasets_AIS[0], bounding_box = bbox, start_date= "5/25/2017 12:00:00 AM", end_date="6/05/2017 11:59:59 PM")
    ais_2017_by_lock_df['date']=ais_2017_by_lock_df['BaseDateTime'].dt.date
    ais_2017_by_lock_df.groupby(['date', 'MMSI']).agg({'BaseDateTime' : {'min_basedate': np.min, 'max_basedate': np.max}})
    ais_2017_by_lock_df['lock_name'] = lock[14]
    df_2017.append(ais_2017_by_lock_df)

    ais_2018_by_lock_df = tm.fetch(datasets_AIS[0], bbox=bbox, start_date="5/25/2018 12:00:00 AM",end_date="6/15/2018 11:59:59 PM")
    ais_2018_by_lock_df['date'] = ais_2018_by_lock_df['BaseDateTime'].dt.date
    ais_2018_by_lock_df.groupby(['date', 'MMSI']).agg({'BaseDateTime': {'min_basedate': np.min, 'max_basedate': np.max}})
    ais_2018_by_lock_df['lock_name'] = lock[14]
    df_2018.append(ais_2018_by_lock_df)

    ais_2019_by_lock_df = tm.fetch(datasets_AIS[0], bbox=bbox, start_date="5/25/2019 12:00:00 AM",end_date="8/15/2019 11:59:59 PM")
    ais_2019_by_lock_df['date'] = ais_2019_by_lock_df['BaseDateTime'].dt.date
    ais_2019_by_lock_df.groupby(['date', 'MMSI']).agg({'BaseDateTime': {'min_basedate': np.min, 'max_basedate': np.max}})
    ais_2019_by_lock_df['lock_name'] = lock[14]
    df_2019.append(ais_2019_by_lock_df)

In [ ]:
df_2017.sort_values(by=['MMSI', 'min_basedate'])
df_2018.sort_values(by=['MMSI', 'min_basedate'])
df_2019.sort_values(by=['MMSI', 'min_basedate'])

In [ ]:
lock_2_lock_2017 = []
travel_time_2017 = []
for i in range(df_2017.shape[0]):
    if (i>0) & (df_2017.at[i-1, 'lock_name'] != df_2017.at[i, 'lock_name']):
        lock_2_lock_2017.append(df_2017.at[i-1, 'lock_name'] + "_" + df_2017.at[i, 'lock_name'])
        travel_time_2017.append(df_2017.at[i, 'BaseDateTime'] - df_2017.at[i-1, 'BaseDateTime'])
    else:
        lock_2_lock_2017.append("")
        travel_time_2017.append("")

In [ ]:
lock_2_lock_2018 = []
travel_time_2018 = []
for i in range(df_2018.shape[0]):
    if (i>0) & (df_2018.at[i-1, 'lock_name'] != df_2018.at[i, 'lock_name']):
        lock_2_lock_2018.append(df_2018.at[i-1, 'lock_name'] + "_" + df_2018.at[i, 'lock_name'])
        travel_time_2018.append(df_2018.at[i, 'BaseDateTime'] - df_2018.at[i-1, 'BaseDateTime'])
    else:
        lock_2_lock_2018.append("")
        travel_time_2018.append("")

In [ ]:
lock_2_lock_2019 = []
travel_time_2019 = []
for i in range(df_2019.shape[0]):
    if (i>0) & (df_2019.at[i-1, 'lock_name'] != df_2019.at[i, 'lock_name']):
        lock_2_lock_2019.append(df_2019.at[i-1, 'lock_name'] + "_" + df_2019.at[i, 'lock_name'])
        travel_time_2019.append(df_2019.at[i, 'BaseDateTime'] - df_2019.at[i-1, 'BaseDateTime'])
    else:
        lock_2_lock_2019.append("")
        travel_time_2019.append("")

In [ ]:
df_2017['lock_2_lock'] = lock_2_lock_2017
df_2018['lock_2_lock'] = lock_2_lock_2018
df_2019['lock_2_lock'] = lock_2_lock_2019

In [ ]:
df_2017['travel_time'] = travel_time_2017
df_2018['travel_time'] = travel_time_2018
df_2019['travel_time'] = travel_time_2019

In [ ]:
df_2017_mean = df_2017.groupby("lock_2_loc")["travel_time"].mean()
df_2018_mean = df_2018.groupby("lock_2_lock")["travel_time"].mean()
df_2019_mean = df_2019.groupby("lock_2_lock")["travel_time"].mean()

In [ ]:
df_matrix_2017 = pd.DataFrame(index=Arkansas_locks['pmsname'], columns=Arkansas_locks['pmsname'])
df_matrix_2018 = pd.DataFrame(index=Arkansas_locks['pmsname'], columns=Arkansas_locks['pmsname'])
df_matrix_2019 = pd.DataFrame(index=Arkansas_locks['pmsname'], columns=Arkansas_locks['pmsname'])

In [ ]:
for i, row in df_matrix_2017:  #row
    for j, row in df_matrix_2017: # col
        name = "".join([i,"_", j])
        if (name in df_2017_mean['lock_2_loc'].values):
            df_matrix_2017[j][i] = df_2017_mean.loc[df_2017_mean['lock_2_loc'] == name, 'travel_time'].iloc[0]